# Trading Report — Predicciones y Backtest
Este notebook explora `data/preds/*` y construye curvas de capital y métricas por modelo y ticker.


In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from models.backtest import backtest_signals, summarize_backtest
import glob, re
pred_files = sorted(glob.glob('data/preds/*.csv'))
len(pred_files), pred_files[:5]

In [ ]:
rows = []
for f in pred_files:
    # parse ticker and model from filename pattern: TICKER_model_splitX.csv
    name = Path(f).stem
    parts = name.split('_')
    ticker = parts[0]
    model = '_'.join(parts[1:-1]) if 'split' in parts[-1] else '_'.join(parts[1:])
    df_bt = backtest_signals(f, threshold=0.0, kind='ret')
    summary = summarize_backtest(df_bt)
    rows.append({'ticker': ticker, 'model': model, **summary})
summary_df = pd.DataFrame(rows)
summary_df.sort_values(['ticker','sharpe_252'], ascending=[True, False])

In [ ]:
if not summary_df.empty:
    for t in summary_df['ticker'].unique():
        top = summary_df[summary_df['ticker']==t].sort_values('sharpe_252', ascending=False).head(5)
        display(top)
else:
    print('No hay predicciones aún en data/preds/. Corre el entrenamiento primero.')

In [ ]:
# Ejemplo de trazado de equity de un archivo específico
if pred_files:
    f = pred_files[0]
    from models.backtest import backtest_signals
    df_bt = backtest_signals(f, threshold=0.0, kind='ret')
    plt.figure()
    plt.plot(df_bt['Datetime'], df_bt['equity'])
    plt.title(f'Equity curve: {Path(f).name}')
    plt.xlabel('Fecha')
    plt.ylabel('Equity')
    plt.tight_layout()
    plt.show()
else:
    print('No hay archivos en data/preds para graficar.')